## Why I use jupyter

It's too boring to train a model on terminal

So I just decided to use jupyter to simplify work

## Let's get started

#### 1. Import my packages

In [1]:
import os
from auto_everything.base import IO, Terminal
io = IO()
t = Terminal()

print(t.run_command('ls'))

collect_json_data.py
data
js
pose_detect_model.h5
py
README.md
start_traning.ipynb


#### 2. Get json files from local storege

In [11]:
files = os.listdir("data")
files = [file for file in files if '.json' in file]
Classes_Num = len(files)

print(files)
print(Classes_Num)

['sitting.json', 'standing.json']
2


#### 3. define a function for calculate distance of raw points

In [12]:
from keras.utils import normalize
import numpy as np
import math

# calculate distance ratio of (0,0) to (x,y)
def prepare_python_list(list_):
    result_frames = []
    for old_frame in list_:
        new_frame = normalize(np.array(old_frame), axis=0).tolist()
        result_frames.append(new_frame)
    return result_frames

#### 4. try to get `x` and `y` (numpy array)

In [13]:
import json

global x, y

for index, file in enumerate(files):
    x_part = json.loads(io.read("data/{name}".format(name=file)))
    x_part = prepare_python_list(x_part)
    x_part = np.array(x_part)

    y_part = np.full(x_part.shape[0], index)

    if index == 0:
        x = x_part
        y = y_part
    else:
        x = np.append(x, x_part, axis=0)
        y = np.append(y, y_part, axis=0)

#### 5. reshape `array` to let it fit the need of `keras`

In [14]:
# randomnize data
index = np.arange(x.shape[0])
np.random.shuffle(index)
print(x,y)

# 3D to 2D
x = x[index]
x = x.reshape(len(x), -1)

y = y[index]

[[[0.22963766 0.26492816]
  [0.22134579 0.24136695]
  [0.23121999 0.20769217]
  ...
  [0.24597837 0.18380082]
  [0.25163442 0.18418821]
  [0.24218532 0.18878434]]

 [[0.24346178 0.27612885]
  [0.23314056 0.24371546]
  [0.2510517  0.20678866]
  ...
  [0.27395422 0.20457598]
  [0.         0.        ]
  [0.27171183 0.20997395]]

 [[0.29406854 0.29685178]
  [0.2945674  0.26583294]
  [0.30750006 0.22478359]
  ...
  [0.         0.        ]
  [0.         0.        ]
  [0.         0.        ]]

 ...

 [[0.22307112 0.25199501]
  [0.21751359 0.2086644 ]
  [0.22869747 0.16649322]
  ...
  [0.24803236 0.12347884]
  [0.25224711 0.1243278 ]
  [0.24461761 0.12972544]]

 [[0.22568805 0.24453686]
  [0.22884192 0.22068287]
  [0.2278397  0.16366805]
  ...
  [0.24789122 0.12085618]
  [0.25160782 0.11668485]
  [0.244867   0.12854136]]

 [[0.22550998 0.24124613]
  [0.23093988 0.21678471]
  [0.22936862 0.16089518]
  ...
  [0.24824484 0.11785668]
  [0.25190319 0.11353664]
  [0.24622657 0.1255766 ]]] [0 0 0 ...

#### 6. build keras model

In [19]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(17, input_dim=x.shape[1], activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(Classes_Num, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 7. papare data, and fit it to model

In [24]:
from keras.utils.np_utils import to_categorical

#y = to_categorical(y)
print(y)

x_train = x[:-50]
y_train = y[:-50]
x_test = x[-50:]
y_test = y[-50:]

model.fit(x_train, y_train, batch_size=10, epochs=100)

[1 1 0 ... 0 1 0]
Epoch 1/100
2470/2470 [==============================] - 0s 98us/step - loss: 0.0986 - acc: 0.9668
Epoch 2/100
2470/2470 [==============================] - 0s 90us/step - loss: 0.0750 - acc: 0.9737
Epoch 3/100
2470/2470 [==============================] - 0s 89us/step - loss: 0.0823 - acc: 0.9688
Epoch 4/100
2470/2470 [==============================] - 0s 93us/step - loss: 0.0906 - acc: 0.9656
Epoch 5/100
2470/2470 [==============================] - 0s 93us/step - loss: 0.0946 - acc: 0.9628
Epoch 6/100
2470/2470 [==============================] - 0s 91us/step - loss: 0.1074 - acc: 0.9575
Epoch 7/100
2470/2470 [==============================] - 0s 94us/step - loss: 0.0951 - acc: 0.9644
Epoch 8/100
2470/2470 [==============================] - 0s 89us/step - loss: 0.0925 - acc: 0.9615
Epoch 9/100
2470/2470 [==============================] - 0s 88us/step - loss: 0.1142 - acc: 0.9534
Epoch 10/100
2470/2470 [==============================] - 0s 91us/step - loss: 0.1210 - acc

#### 8. test

In [25]:
loss, accuracy = model.evaluate(x_test, y_test)
print('loss:', loss)
print('accuracy', accuracy)


print('\n'*2)
print(y_test)
predictions = model.predict(x_test)
predictions = [np.argmax(element) for element in predictions]
predictions = np.array(predictions)
print(predictions)

50/50 [==============================] - 0s 43us/step
loss: 0.07360834233462811
accuracy 0.98



[0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 0 0 1
 1 1 1 1 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 0 0 1
 1 1 1 1 1 0 0 0 0 0 0 1 0]


#### 9. save model

In [26]:
model.save('pose_detect_model.h5')

#### 10. convert keras model to tensorflowjs

In [27]:
c = "tensorflowjs_converter --input_format=keras --output_format=tensorflowjs pose_detect_model.h5 ../../standing_or_sitting_models"
r = t.run_command(c)
print(r)

/home/yingshaoxo/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
